`detailed_node_and_edge_features.ipynb`의 주된 역할은 다음과 같습니다:

1. **OSM으로부터 그래프 불러오기**  
   - 특정 도시의 도로망을 OSMnx로 로드해 GeoDataFrame(`gdf_nodes`, `gdf_edges`) 형태로 얻습니다.

2. **사고 데이터와 그래프 노드 매핑**  
   - US-Accidents 데이터의 위경도 좌표를 `nearest_nodes()`로 가장 가까운 OSM 노드에 매핑해, 각 노드에 사고 건수(`accident_cnt`)와 평균 심각도(`severity`) 정보를 연결합니다.

3. **노드 특성 추출 및 전처리**  
   - 논문 Table 2에 정의된 노드 특성들(`highway`, `street_count` 등)을 선택하고, `get_dummies`로 one-hot 인코딩해 `features` 매트릭스를 만듭니다.
   - 최종적으로 모델 입력용 노드 피처 행렬의 크기(shape)와 컬럼명을 화면에 출력해서 “이렇게 생겼다”고 검증합니다.

4. **엣지 특성 추출 및 전처리**  
   - 엣지 GeoDataFrame에서 `length`, `lanes`, `maxspeed`, `bridge`, `oneway`, `highway` 등을 선별하고, 결측치·리스트 타입을 적절히 처리한 뒤 one-hot 인코딩합니다.
   - `edge_attrs` 매트릭스로 만들어 모델 입력용 엣지 피처 행렬의 크기와 컬럼 구성을 확인합니다.

5. **디버깅 및 검증**  
   - “nearest_nodes 버그 체크”, 불완전한 카테고리 처리, 누락된 속성 예외 처리 로직 등을 포함해, 실제 전처리 코드가 논문에 맞게 정확히 동작하는지 단계별로 점검합니다.

요약하면, 이 노트북은 논문의 Section 4 “데이터 전처리” 중 “어떤 지리공간(geo-spatial) 피처가 노드/엣지 특성 행렬로 대응되는지”를 실제 코드로 추출·시각화·검증하는 역할을 합니다.

In [ ]:
from pathlib import Path
import pandas as pd
import ujson as json   # 빠른 JSON 파싱
import os

base = Path('data')

# ── 1) 모든 CSV/JSON 경로 한 번에 수집 ───────────────────────────────
csv_paths  = list(base.rglob('*.csv'))
json_paths = list(base.rglob('*.json'))

# ── 2) V2X 로그 (CSV) 일괄 처리 ────────────────────────────────────
# 실제 분석에 필요한 컬럼만 지정
usecols = [
    'ISSUE_DATE','LONGITUDE','LATITUDE','HEADING',
    'SPEED','BRAKE_STATUS','ACC_SEC',
    'CURRENT_LANE','VEHICLE_TYPE'
]

v2x_dfs = []
for p in csv_paths:
    # 경로 조각: ('9월','220911','C','A','9CD0DFDC','V_220911_C_A_9CD0DFDC_0001.csv')
    parts = p.relative_to(base).parts
    if len(parts) < 5:
        continue

    _, date_str, loc, tt, dev = parts[:5]
    try:
        date = pd.to_datetime(date_str, format='%y%m%d')
    except ValueError:
        continue

    # 필요한 컬럼만 읽어들임
    df = pd.read_csv(p, usecols=usecols, parse_dates=['ISSUE_DATE'])
    df['Date']      = date
    df['Location']  = loc
    df['TimeType']  = tt
    df['Device_ID'] = dev
    v2x_dfs.append(df)

if v2x_dfs:
    v2x_df = pd.concat(v2x_dfs, ignore_index=True)
    print("◾ V2X logs:", v2x_df.shape)
else:
    print("⚠️ V2X logs 비어 있음!")

# ── 3) Annotations (JSON) 일괄 처리 ─────────────────────────────────
anno_recs = []
for p in json_paths:
    parts = p.relative_to(base).parts
    if len(parts) < 5:
        continue

    _, date_str, loc, tt, dev = parts[:5]
    try:
        date = pd.to_datetime(date_str, format='%y%m%d')
    except ValueError:
        continue

    with open(p, 'r', encoding='utf-8') as fp:
        j = json.load(fp)
    anno_recs.append({
        'Date':      date,
        'Location':  loc,
        'TimeType':  tt,
        'Device_ID': dev,
        'Turn':      j['Annotation']['Turn'],
        'Lane':      j['Annotation']['Lane'],
        'SpeedEvt':  j['Annotation']['Speed'],
        'Hazard':    j['Annotation']['Hazard'],
    })

anno_df = pd.DataFrame(anno_recs)
print("◾ Annotations:", anno_df.shape)


In [ ]:
!pip install osmnx

# 노드 및 엣지 특성
- 이 노트북은 TAP(교통사고 예측)에서 사용되는 노드 특성 행렬에 대응하는 지리공간(geo-spatial) 특징들이 무엇인지 추가 정보를 제공합니다.


In [ ]:
import numpy as np
import pandas as pd
import osmnx as ox
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data

np.random.seed(17)
print("OSMnx version:", ox.__version__)

OSMnx version: 2.0.2


In [ ]:
# 1) V2X 로그 불러오기
v2x = pd.read_csv('datasets/v2x_logs.csv', parse_dates=['ISSUE_DATE'])

In [ ]:
# 2) 좌표 → segment_id 매핑 함수
def map_to_segment(lon, lat, G):
    return ox.distance.nearest_edges(G, X=lon, Y=lat)

In [ ]:
# 3) 지역별 처리 (광주/세종)
for loc_code, place_name in {'C':'Gwangju, South Korea','S':'Sejong, South Korea'}.items():
    print(f"=== {loc_code} ({place_name}) ===")
    G_city = ox.graph_from_place(place_name, simplify=True, network_type='drive')
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_city)

    # 4) 노드 static 피처
    node_static = gdf_nodes[['x','y','street_count','highway']].copy()
    node_static['highway'].fillna('nan', inplace=True)
    node_static = pd.concat([
        node_static.drop('highway',axis=1),
        pd.get_dummies(node_static.highway, prefix='hw')
    ], axis=1)
    x = torch.tensor(node_static.values, dtype=torch.float)
    print(" node_features:", x.shape)

In [ ]:
    # 5) V2X 동적 엣지 피처
    v2x['segment_id'] = v2x.apply(
        lambda r: map_to_segment(r.LONGITUDE, r.LATITUDE, G_city),
        axis=1
    )
    agg = {
      'SPEED': ['mean','var'],
      'BRAKE_STATUS': 'mean',
      'ACC_SEC': ['mean','max','min'],
      'CURRENT_LANE': lambda x: x.diff().abs().fillna(0).sum()
    }
    dyn = v2x.groupby('segment_id').agg(agg)
    dyn.columns = ['_'.join(c) for c in dyn.columns]
    dyn = dyn.fillna(0)

Current City: ('New York', 'New York')
Node shape: (55292, 12) features: Index(['street_count', 'high_crossing', 'high_give_way',
       'high_motorway_junction', 'high_nan', 'high_priority', 'high_stop',
       'high_toll_gantry', 'high_traffic_signals',
       'high_traffic_signals;crossing', 'high_turning_circle',
       'high_turning_loop'],
      dtype='object')
Edge shape: (139463, 19) features: Index(['length', 'bridge', 'lanes', 'oneway_False', 'oneway_True',
       'high_busway', 'high_living_street', 'high_motorway',
       'high_motorway_link', 'high_primary', 'high_primary_link',
       'high_residential', 'high_secondary', 'high_secondary_link',
       'high_tertiary', 'high_tertiary_link', 'high_trunk', 'high_trunk_link',
       'high_unclassified'],
      dtype='object')


In [ ]:
    # 6) Static(OSM) + Dynamic(V2X) 병합
    ed = gdf_edges.set_index(['u','v','key']).join(dyn, how='left').fillna(0)
    static_attrs  = ['highway','oneway','length','bridge','lanes']
    dynamic_attrs = list(dyn.columns)
    ed = ed[static_attrs + dynamic_attrs]

In [ ]:
    # 7) 후처리
    if 'bridge' in ed:
        ed['bridge'] = LabelEncoder().fit_transform(ed['bridge'].fillna('nan'))
    if 'lanes' in ed:
        ed['lanes'] = LabelEncoder().fit_transform(ed['lanes'].astype(str).fillna('-1'))
    ed = pd.concat([
        ed,
        pd.get_dummies(ed.highway, prefix='hw'),
        pd.get_dummies(ed.oneway, prefix='ow')
    ], axis=1)

In [ ]:
    # 8) PyG Data
    n2i = {nid:i for i,nid in enumerate(gdf_nodes.index)}
    ei = torch.tensor([[n2i[u] for u,v in ed.index],
                       [n2i[v] for u,v in ed.index]], dtype=torch.long)
    ea = torch.tensor(ed.drop(['highway','oneway'],axis=1).values, dtype=torch.float)
    data = Data(x=x, edge_index=ei, edge_attr=ea)
    print(" edge_attr:", ea.shape,"\n", data)